# Crunch 1 - Exploratory Analysis for Spatial Transcriptomics Prediction

## Purpose of This Notebook
This notebook is intended to facilitate an exploratory analysis of the data provided for **Crunch 1** of the Autoimmune Disease Machine Learning Challenge. The purpose of this analysis is to:
- Understand the structure and characteristics of the datasets.
- Validate the integrity and alignment of the data.
- Identify patterns and relationships between the input data (`X`: H&E images) and output data (`Y`: spatial transcriptomics).
- Provide a foundation for preprocessing steps, feature engineering, and model development.

Exploratory data analysis (EDA) involves systematically reviewing and visualizing data to uncover meaningful insights. It is a critical step in any data science workflow, particularly when working with complex datasets like those in this challenge.

## Objectives

1. **Dataset Inspection**:
   - The structure and contents of the datasets will be examined. This includes reviewing `HE_registered`, `HE_original`, `anucleus`, and `transcripts`.
   - Potential issues, such as missing data or misaligned images, will be identified.

2. **Visualization**:
   - The spatial organization of nuclei and gene expression in tissue regions will be explored.
   - H&E image data will be visualized alongside transcriptomics data to understand how these modalities relate to each other spatially.

3. **Statistical Analysis**:
   - Distributions of gene expression and image features will be summarized.
   - Correlations and patterns between the input (`X`) and output (`Y`) datasets will be analyzed to guide the development of features for modeling.

## Inputs and Outputs
### **Inputs (`X`)**
1. **HE_registered**: These are H&E pathology images that have been registered to the coordinate system used by the spatial transcriptomics data. Registration ensures that the images are aligned with the gene expression measurements and simplifies downstream analysis. This dataset is recommended as it avoids the need for additional alignment.
2. **HE_original**: These are the unaligned H&E images in their native pixel coordinates. While alignment may need to be performed manually, these images could provide additional flexibility for advanced preprocessing tasks.

### **Outputs (`Y`)**
1. **anucleus**: This dataset contains aggregated gene expression profiles for 460 genes, with one profile for each nucleus. The gene expression data has been log1p-normalized, a transformation that makes the data more manageable for machine learning by compressing large values and emphasizing smaller ones.
2. **transcripts**: This dataset provides raw spatial transcriptomics data, including the expression levels of individual genes and their spatial locations within the tissue. These data are linked to specific nuclei and are aggregated to produce `anucleus`.

## Success Metrics
The goal of this analysis is to ensure that the datasets are ready for use in developing a predictive model. The ultimate objective of the challenge is to predict gene expression in regions of tissue where this information is held out.

- **End Goal**: The prediction of log1p-normalized gene expression for 460 genes in held-out regions of the tissue.
- **Evaluation Metric**: The predictions will be assessed using the Mean Squared Error (MSE), which measures the average squared difference between the predicted and actual gene expression values.

By the conclusion of this notebook, the datasets will have been thoroughly explored, and actionable insights will have been generated to inform preprocessing, feature engineering, and model development.


In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import spatialdata_plot
import spatialdata as sd
import scanpy as sc

/home/secondbook5/micromamba/envs/autoimmune_ml/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [3]:
from src.utils.config_loader import ConfigLoader
from pathlib import Path

# Step 1: Initialize the ConfigLoader with the relative path to config.yaml
config_loader = ConfigLoader("../../config.yaml")

# Step 2: Retrieve the raw_dir for Crunch 1 from the configuration
raw_dir = config_loader.get_crunch_path("crunch1", "raw_dir")

# Step 3: Construct the full path to the .zarr file
zarr_file_path = Path(raw_dir) / "UC1_NI.zarr"

# Step 4: Load the .zarr file using spatialdata
sdata = sd.read_zarr(str(zarr_file_path))

# Step 5: Output the spatial data
print(sdata)

SpatialData object, with associated Zarr store: /mnt/d/AutoImmuneML/broad-1-autoimmune-crunch1/data/UC1_NI.zarr
├── Images
│     ├── 'HE_nuc_original': DataArray[cyx] (1, 21000, 22000)
│     └── 'HE_original': DataArray[cyx] (3, 21000, 22000)
└── Tables
      ├── 'anucleus': AnnData (80037, 460)
      └── 'cell_id-group': AnnData (93686, 0)
with coordinate systems:
    ▸ 'global', with elements:
        HE_nuc_original (Images), HE_original (Images)


In [4]:
print("[INFO] Available image keys:", sdata.images.keys())


[INFO] Available image keys: KeysView({'HE_nuc_original': <xarray.DataArray 'image' (c: 1, y: 21000, x: 22000)> Size: 2GB
dask.array<from-zarr, shape=(1, 21000, 22000), dtype=uint32, chunksize=(1, 5792, 5792), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 8B 0
  * y        (y) float64 168kB 0.5 1.5 2.5 3.5 ... 2.1e+04 2.1e+04 2.1e+04
  * x        (x) float64 176kB 0.5 1.5 2.5 3.5 ... 2.2e+04 2.2e+04 2.2e+04
Attributes:
    transform:  {'global': Identity }, 'HE_original': <xarray.DataArray 'image' (c: 3, y: 21000, x: 22000)> Size: 1GB
dask.array<from-zarr, shape=(3, 21000, 22000), dtype=uint8, chunksize=(3, 6688, 6688), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 24B 0 1 2
  * y        (y) float64 168kB 0.5 1.5 2.5 3.5 ... 2.1e+04 2.1e+04 2.1e+04
  * x        (x) float64 176kB 0.5 1.5 2.5 3.5 ... 2.2e+04 2.2e+04 2.2e+04
Attributes:
    transform:  {'global': Identity }})


## Input (X) Data: H&E Images

This function takes a list of image keys and titles renders the images using SpatialData's visualization capabilities, and arranges them in a single row of subplots for side-by-side comparison.

The visualization works by rasterizing the data stored in the SpatialData object, which involves converting high-resolution vector-based spatial information into raster images that can be displayed on the screen.
This is essential for exploring large tissue images, ensuring that the details are clearly visible while keeping memory usage manageable.

How It Works:
- The function creates a grid of subplots, each corresponding to an image key.
- For each image key, the SpatialData object's `.pl.render_images` method is used to rasterize and display the image. This ensures compatibility with large images.
- Titles are added to the subplots for clarity.
- `tight_layout()` is called to prevent overlapping of subplot elements.
### 2.1 Visualizing H&E Image

In [ ]:
# Visualize the H&E images
def visualize_he_images(sdata, image_keys, titles, figsize=(20, 10)):
    """
    Visualize H&E images using SpatialData.

    Args:
        sdata (SpatialData): The spatial data object containing the images.
        image_keys (list): List of keys for the images to visualize (e.g., ["HE_nuc_original", "HE_original"]).
        titles (list): List of titles corresponding to the images.
        figsize (tuple): Figure size for the plots.
    """
    # Validate inputs
    if len(image_keys) != len(titles):
        raise ValueError("Length of image_keys must match length of titles.")

    # Create subplots
    fig, axes = plt.subplots(1, len(image_keys), figsize=figsize)
    axes = axes.flatten()

    for ax, key, title in zip(axes, image_keys, titles):
        # Render the image and show it in the subplot
        sdata.pl.render_images(key).pl.show(ax=ax, title=title, coordinate_systems="global")

    plt.tight_layout()
    plt.show()

# Example usage for the quickstarter images
visualize_he_images(
    sdata,
    image_keys=["HE_nuc_original", "HE_original"],
    titles=["H&E Registered Image", "H&E Original Image"]
)


INFO     Rasterizing image for faster rendering.                                                                   


#### 2.1.1 Analyze Pixel Intensities

In [ ]:
def analyze_pixel_intensities(sdata, keys):
    """
    Analyze pixel intensities and display their distribution for images.

    Args:
        sdata (SpatialData): The spatial data object.
        keys (list): Keys of the images to analyze.
    """
    for key in keys:
        # Extract the image as a NumPy array
        image = sdata.images[key].to_numpy()

        # Calculate statistics
        mean_intensity = image.mean()
        median_intensity = np.median(image)
        std_dev = image.std()
        min_intensity = image.min()
        max_intensity = image.max()
        shape = image.shape

        # Log statistics
        print(f"[INFO] Image '{key}':")
        print(f"  - Shape: {shape}")
        print(f"  - Mean Intensity: {mean_intensity:.2f}")
        print(f"  - Median Intensity: {median_intensity:.2f}")
        print(f"  - Standard Deviation: {std_dev:.2f}")
        print(f"  - Min Intensity: {min_intensity}")
        print(f"  - Max Intensity: {max_intensity}")

        # Plot histogram of pixel intensities
        plt.figure(figsize=(10, 6))
        plt.hist(image.flatten(), bins=100, color="blue", alpha=0.7, log=True)
        plt.title(f"Pixel Intensity Distribution: {key}", fontsize=14, fontweight="bold")
        plt.xlabel("Pixel Intensity")
        plt.ylabel("Frequency (log scale)")
        plt.grid(True, linestyle='--', linewidth=0.5)
        plt.show()

# Example usage
analyze_pixel_intensities(
    sdata,
    keys=["HE_nuc_original", "HE_original"]
)


### 2.2 Nuclei Overlay and Feature Analysis

In this section, we focus on analyzing nuclei features and overlaying their centroids on H&E images. The following tasks are performed to gain insights into the distribution and characteristics of nuclei in the dataset.

#### 2.2.1 Nuclei Overlay on H&E Images
This step involves overlaying the centroids of nuclei on the corresponding H&E image. This visualization helps to understand the spatial distribution of nuclei within the tissue.

**Code Explanation:**
- Validate the existence of required image keys in the dataset.
- Extract the original H&E image and nuclei mask using the specified keys.
- Compute centroids of the nuclei using `regionprops` from `skimage`.
- Plot the H&E image and overlay the nuclei centroids using scatter points.

**Results:**
- The overlay visualization confirms the spatial arrangement and density of nuclei across the tissue.
- The output highlights how nuclei are distributed in relation to the tissue structure.

In [ ]:
from skimage.measure import regionprops

def plot_nuclei_overlay(sdata, image_key, nuclei_key="HE_nuc_original", marker_color="yellow", marker_size=15):
    """
    Overlay nuclei centroids on the H&E image.

    Args:
        sdata (SpatialData): The spatial data object.
        image_key (str): Key of the H&E image.
        nuclei_key (str): Key of the nuclei image.
        marker_color (str): Color of the markers for nuclei centroids.
        marker_size (int): Size of the markers for nuclei centroids.
    """
    # Validate input keys
    if image_key not in sdata.images.keys():
        raise KeyError(f"Image key '{image_key}' not found in SpatialData images.")
    if nuclei_key not in sdata.images.keys():
        raise KeyError(f"Nuclei key '{nuclei_key}' not found in SpatialData images.")

    # Load images
    image = sdata.images[image_key].to_numpy()
    nuclei = sdata.images[nuclei_key].to_numpy()

    # Handle image dimensions
    if image.shape[0] == 1:  # Grayscale with singleton channel
        image = image.squeeze(0)
    elif image.shape[0] == 3:  # RGB with three channels
        image = np.transpose(image, (1, 2, 0))

    if nuclei.shape[0] == 1:  # Grayscale nuclei image
        nuclei = nuclei.squeeze(0)

    # Extract nuclei centroids
    regions = regionprops(nuclei)
    centroids = [r.centroid for r in regions]

    if not centroids:
        print("[INFO] No nuclei centroids found in the provided nuclei image.")
        return

    # Plot the H&E image with overlay
    plt.figure(figsize=(12, 12))
    plt.imshow(image, cmap="gray" if image.ndim == 2 else None)  # Grayscale if 2D, otherwise RGB
    plt.scatter(
        [c[1] for c in centroids], [c[0] for c in centroids],
        c=marker_color, s=marker_size, label="Nuclei Centroids"
    )
    plt.title(f"Overlay of Nuclei on {image_key}", fontsize=16, fontweight="bold")
    plt.legend()
    plt.axis("off")
    plt.show()

# Example usage
plot_nuclei_overlay(
    sdata, image_key="HE_original", nuclei_key="HE_nuc_original",
    marker_color="red", marker_size=20
)


#### 2.2.2 Nuclei Feature Distribution and Analysis
In this step, we analyze key features of the nuclei such as their **area** and **eccentricity**. The analysis is visualized through histograms for easier interpretation.

**Code Explanation:**
- Compute the **area** and **eccentricity** of each nucleus using the `regionprops` method.
- Display summary statistics, including the total number of nuclei, mean, and median values for both area and eccentricity.
- Plot histograms to visualize the distributions of area and eccentricity.

**Key Metrics:**
- **Area**: Represents the size of each nucleus.
  - Mean Area: **46.24**
  - Median Area: **44.00**
- **Eccentricity**: Describes the elongation of the nucleus (value between 0 and 1, where 0 indicates a perfect circle).
  - Mean Eccentricity: **0.68**
  - Median Eccentricity: **0.71**

**Results:**
- The area distribution indicates most nuclei are between 30-70 units in size.
- Eccentricity distribution shows a bias towards higher eccentricity, suggesting elongated nuclei are common in this dataset.

---

**Figures:**
1. **Nuclei Area Distribution**:
   - A histogram showing the frequency of nuclei sizes.
   - Majority of nuclei fall within the 30-70 range, with a tail extending to larger sizes.

2. **Nuclei Eccentricity Distribution**:
   - A histogram displaying the frequency of eccentricity values.
   - The distribution peaks around 0.6-0.8, showing a prevalence of moderately elongated nuclei.

In [ ]:
from skimage.measure import regionprops, label

def analyze_nuclei_features(nuclei_image):
    """
    Analyze nuclei features such as area and eccentricity.

    Args:
        nuclei_image (np.ndarray): Binary nuclei image.

    Returns:
        None
    """
    # Label nuclei regions
    labeled_nuclei = label(nuclei_image)
    regions = regionprops(labeled_nuclei)

    areas = [r.area for r in regions]
    eccentricities = [r.eccentricity for r in regions]

    # Summary statistics
    print("\n[INFO] Nuclei Feature Analysis:")
    print(f" - Total Nuclei: {len(regions)}")
    print(f" - Mean Area: {np.mean(areas):.2f}")
    print(f" - Median Area: {np.median(areas):.2f}")
    print(f" - Mean Eccentricity: {np.mean(eccentricities):.2f}")
    print(f" - Median Eccentricity: {np.median(eccentricities):.2f}")

    # Plot histograms
    plt.figure(figsize=(10, 6))
    plt.hist(areas, bins=50, color="blue", alpha=0.7)
    plt.title("Nuclei Area Distribution", fontsize=14, fontweight="bold")
    plt.xlabel("Area")
    plt.ylabel("Frequency")
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.hist(eccentricities, bins=50, color="green", alpha=0.7)
    plt.title("Nuclei Eccentricity Distribution", fontsize=14, fontweight="bold")
    plt.xlabel("Eccentricity")
    plt.ylabel("Frequency")
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.show()

# Example usage
nuclei_image = sdata.images["HE_nuc_original"].to_numpy().squeeze(0)
analyze_nuclei_features(nuclei_image)


### **2.3 Generate a Nuclei Density Heatmap**

A nuclei density heatmap provides insights into the spatial clustering and density of nuclei across the tissue section. This can be a valuable intermediate step for understanding tissue architecture.

---

#### **2.3.1 Nuclei Density Heatmap**
This function will:
1. Extract the centroids of nuclei from the binary nuclei image.
2. Compute the density using kernel density estimation (KDE).
3. Visualize the heatmap using `matplotlib`.


In [ ]:
from scipy.stats import gaussian_kde

def generate_nuclei_density_heatmap(nuclei_image, bins=100):
    """
    Generate and display a nuclei density heatmap.

    Args:
        nuclei_image (np.ndarray): Binary nuclei image.
        bins (int): Number of bins for the heatmap.
    """
    # Extract nuclei centroids
    labeled_nuclei = label(nuclei_image)
    regions = regionprops(labeled_nuclei)
    centroids = np.array([region.centroid for region in regions])

    if centroids.size == 0:
        print("[WARNING] No nuclei detected. Skipping heatmap generation.")
        return

    # Perform kernel density estimation (KDE)
    kde = gaussian_kde(centroids.T)
    x = np.linspace(0, nuclei_image.shape[1], bins)
    y = np.linspace(0, nuclei_image.shape[0], bins)
    X, Y = np.meshgrid(x, y)
    Z = kde(np.vstack([Y.ravel(), X.ravel()])).reshape(X.shape)

    # Plot the heatmap
    plt.figure(figsize=(10, 10))
    plt.imshow(Z, extent=(0, nuclei_image.shape[1], nuclei_image.shape[0], 0), cmap="hot", alpha=0.7)
    plt.colorbar(label="Nuclei Density")
    plt.title("Nuclei Density Heatmap", fontsize=16, fontweight="bold")
    plt.axis("off")
    plt.show()

# Example usage
nuclei_image = sdata.images["HE_nuc_original"].to_numpy().squeeze(0)
generate_nuclei_density_heatmap(nuclei_image, bins=200)

In [ ]:
from skimage.measure import regionprops, label
from scipy.stats import gaussian_kde
from matplotlib.colors import LogNorm
from joblib import Parallel, delayed

def enhanced_density_heatmap(nuclei_image, bins=200, contour_levels=10):
    """
    Generate a high-resolution density heatmap with optimized KDE.

    Args:
        nuclei_image (np.ndarray): Binary nuclei image.
        bins (int): Number of bins for the heatmap.
        contour_levels (int): Number of contour levels.
    """
    # Step 1: Extract centroids
    labeled_nuclei = label(nuclei_image)
    regions = regionprops(labeled_nuclei)
    centroids = np.array(
        Parallel(n_jobs=-1)(
            delayed(lambda r: r.centroid)(region) for region in regions
        )
    )

    if centroids.size == 0:
        print("[WARNING] No nuclei detected. Skipping heatmap generation.")
        return

    # Step 2: Perform KDE
    kde = gaussian_kde(centroids.T)
    x = np.linspace(0, nuclei_image.shape[1], bins)
    y = np.linspace(0, nuclei_image.shape[0], bins)
    X, Y = np.meshgrid(x, y)
    Z = kde(np.vstack([Y.ravel(), X.ravel()])).reshape(X.shape)

    # Step 3: Plot heatmap and contours
    plt.figure(figsize=(12, 12))
    plt.imshow(Z, extent=(0, nuclei_image.shape[1], nuclei_image.shape[0], 0),
               cmap="hot", norm=LogNorm(vmin=1e-2, vmax=Z.max()), alpha=0.8)
    plt.colorbar(label="Nuclei Density")

    # Overlay contour lines
    plt.contour(X, Y, Z, levels=contour_levels, colors="cyan", alpha=0.7, linewidths=0.5)
    plt.title("Enhanced Nuclei Density Heatmap", fontsize=16, fontweight="bold")
    plt.axis("off")
    plt.show()

# Example usage
nuclei_image = sdata.images["HE_nuc_original"].to_numpy().squeeze(0)
enhanced_density_heatmap(nuclei_image, bins=200, contour_levels=15)


### 2.3.2 DBSCAN and Density-Based Clustering

#### 2.3.2.1 Basic DBSCAN with k-Distance Graph
In this step, we implement a DBSCAN approach to cluster nuclei. We calculate an optimal epsilon (`eps`) using a k-distance graph and apply the DBSCAN algorithm to the extracted nuclei features.

- **Steps**:
  1. Extract basic nuclei features: centroid, area, and eccentricity.
  2. Use a k-distance graph to determine the optimal epsilon (`eps`).
  3. Apply DBSCAN clustering to the nuclei features.
  4. Evaluate clustering quality using the following metrics:
     - **Silhouette Score**: Measures how similar each point is to its own cluster compared to other clusters.
     - **Davies-Bouldin Index**: Measures the ratio of within-cluster scatter to between-cluster separation.
     - **Calinski-Harabasz Score**: Measures the variance ratio between clusters.

- **Results**:
  - k-Distance Graph: Visual inspection for the elbow point to determine the optimal epsilon value.
  - Initial clustering evaluation metrics to benchmark performance.


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler


def calculate_optimal_eps(features: np.ndarray, k: int = 5):
    """
    Calculate the optimal epsilon using k-distance graph for DBSCAN.

    Args:
        features (np.ndarray): Features for clustering.
        k (int): Number of nearest neighbors.

    Returns:
        float: Suggested value of epsilon based on the elbow method.
    """
    neighbors = NearestNeighbors(n_neighbors=k, n_jobs=-1)
    distances, _ = neighbors.fit(features).kneighbors(features)
    k_distances = np.sort(distances[:, -1])  # Sort distances to k-th neighbor

    # Plot k-distance graph for manual inspection
    plt.figure(figsize=(10, 6))
    plt.plot(k_distances, marker='o', linestyle='--', markersize=3)
    plt.title("k-Distance Graph for DBSCAN (Elbow Method)", fontsize=14, fontweight="bold")
    plt.xlabel("Points (sorted by distance)")
    plt.ylabel(f"Distance to {k}-th Nearest Neighbor")
    plt.grid(True, linestyle="--", linewidth=0.5)
    plt.show()

    # Suggest epsilon at the elbow (90th percentile)
    suggested_eps = np.percentile(k_distances, 90)
    print(f"[INFO] Suggested epsilon (eps): {suggested_eps:.4f}")
    return suggested_eps


def dbscan_with_advanced_metrics(nuclei_image: np.ndarray, min_samples: int = 5, k: int = 5, use_pca: bool = True):
    """
    Perform advanced DBSCAN clustering with full diagnostics and feature engineering.

    Args:
        nuclei_image (np.ndarray): Binary nuclei image.
        min_samples (int): Minimum samples for DBSCAN.
        k (int): Number of neighbors for epsilon estimation.
        use_pca (bool): Whether to use PCA for dimensionality reduction.

    Returns:
        pd.DataFrame: DataFrame with clustering results.
        np.ndarray: Cluster labels.
    """
    # Extract advanced nuclei features
    labeled_nuclei = label(nuclei_image)
    regions = regionprops(labeled_nuclei)
    features = []

    for region in regions:
        compactness = (region.area) / (region.perimeter ** 2) if region.perimeter > 0 else 0
        features.append([region.centroid[0], region.centroid[1], region.area, region.eccentricity, compactness])

    features_df = pd.DataFrame(features, columns=["y", "x", "area", "eccentricity", "compactness"])
    print("[INFO] Extracted features for clustering:")
    display(features_df.head())

    # Normalize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_df)

    # Optional PCA
    if use_pca:
        pca = IncrementalPCA(n_components=2)
        features_reduced = pca.fit_transform(features_scaled)
        print("[INFO] Reduced features to 2 dimensions using PCA.")
    else:
        features_reduced = features_scaled

    # Compute optimal epsilon
    eps = calculate_optimal_eps(features_reduced, k)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    cluster_labels = dbscan.fit_predict(features_reduced)
    features_df["cluster"] = cluster_labels

    # Evaluate clustering performance
    silhouette = silhouette_score(features_reduced, cluster_labels) if len(set(cluster_labels)) > 1 else -1
    db_score = davies_bouldin_score(features_reduced, cluster_labels) if len(set(cluster_labels)) > 1 else -1
    ch_score = calinski_harabasz_score(features_reduced, cluster_labels) if len(set(cluster_labels)) > 1 else -1

    print(f"[INFO] Silhouette Score: {silhouette:.4f}")
    print(f"[INFO] Davies-Bouldin Index: {db_score:.4f} (lower is better)")
    print(f"[INFO] Calinski-Harabasz Score: {ch_score:.4f} (higher is better)")
    return features_df, cluster_labels


# Example usage
dbscan_features_df, dbscan_labels = dbscan_with_advanced_metrics(nuclei_image, min_samples=10, k=10)


#### 2.3.2.2 Advanced DBSCAN with Diagnostics and Feature Engineering
Building upon the basic DBSCAN approach, we now introduce advanced diagnostics and feature engineering to improve clustering performance and interpretability.

- **Enhancements**:
  1. **Feature Engineering**:
     - Add compactness as a feature to better characterize nuclei shapes.
     - Optionally include intensity-based features (mean and standard deviation) for richer data representation.
  2. **Dimensionality Reduction**:
     - Use Incremental PCA to reduce feature dimensionality for large datasets, ensuring computational efficiency.
  3. **Evaluation Metrics**:
     - Evaluate clustering quality using silhouette score, Davies-Bouldin index, and Calinski-Harabasz score.
  4. **Visualization**:
     - Highlight outliers in clustering visualizations.
     - Provide enhanced plots to interpret clusters in the spatial context.

- **Results**:
  - Optimized epsilon and `min_samples` values based on diagnostics.
  - Improved clustering performance metrics.
  - Visualizations showcasing well-defined clusters and detected outliers.

- **Notes**:
  - Further tuning of parameters (e.g., `eps`, `min_samples`) may lead to better results.
  - Combining insights from density analysis (Step 2.3.1) can guide feature engineering and cluster interpretation.
